<h1><center>Bioptim Workshop</center></h1>

# Let's start with a simple example
Please find below a simple example of an inverted pendulum. The goal is, by using only 
side pushes, to bring the wand upward. 

The ocp to solve is as follow:
- Movement of $1s$ separted into $50$ shooting points
- States variables are the generalized position (q) and velocities (qdot)
- Side translation is bounded to $\pm10\ m$, and rotation to $\pm2\pi\ rad$. Velocities are bounded to $\pm10\ m/s$ and $\pm31.4\ rad/s$, respectively
- Control variables are the generalized forces (tau) 
- Side translation force is bounded to $\pm100\ N$ and the rotation force is not allowed (bonded to $0\ Nm$)
- Cost functions: minimize the side pushes forces at all time
- Initial guess is arbitrarily set to $0$ for all the variables

In [ ]:
# So first, let's import all the required classes 
from bioptim import *

In [ ]:
# Now let's define the actual ocp

# Let's load a model with its dynamics and define some aliases
model_path = "models/pendulum.bioMod"
model = TorqueBiorbdModel(model_path)
nq = model.nb_q  # Number of degrees of freedom in the model

# Define the time of movement and number of shooting point
final_time = 1
n_shoot = 100

# Pass some option to the dynamics. Here we ask for an expanded dynamics
# This trades memory (RAM) for speed
dynamics = DynamicsOptions(expand_dynamics=True)

# Define the path constraints of the states (x) and controls (u)
x_bounds = BoundsList()
x_bounds["q"] = model.bounds_from_ranges("q")
x_bounds["q"][:, [0, -1]] = 0  # Start and end at 0...
x_bounds["q"][1, -1] = 3.14  # ...but end with pendulum 180 degrees rotated
x_bounds["qdot"] = model.bounds_from_ranges("qdot")
x_bounds["qdot"][:, [0, -1]] = 0  # Start and end without any velocity

u_bounds = BoundsList()
u_bounds["tau"] = [-100] * nq, [100] * nq  # Limit the strength of the pendulum to (-100 to 100)...
u_bounds["tau"][1, :] = 0  # ...but remove the capability to actively rotate

# Define the objective functions
objective_functions = Objective(ObjectiveFcn.Lagrange.MINIMIZE_CONTROL, key='tau')  # Minimize the generalized forces

# We can define initial guesses here, but if we don't zero is the default
x_init = InitialGuessList()
x_init["q"] = [0] * model.nb_q
x_init["qdot"] = [0] * model.nb_qdot
u_init = InitialGuessList()
u_init["tau"] = [0] * nq

# Send all this to the ocp structure
ocp = OptimalControlProgram(
    model,
    n_shoot,
    final_time,
    dynamics=dynamics,
    x_init=x_init, 
    u_init=u_init, 
    x_bounds=x_bounds,
    u_bounds=u_bounds,
    objective_functions=objective_functions,
)

# Have a look on the problem
ocp.print(to_console=True,to_graph=False)

In [ ]:
# Let's solve the ocp!
sol = ocp.solve()

In [ ]:
# Now let's print some results to the console and plot some graphs
sol.print_cost()
sol.graphs(automatically_organize=False)

In [ ]:
# This cell for visualizing the animation won't work in a jupyter notebook, but would work locally, assuming 
# a viewer ([bioviz] or pyorerun) is properly installed

# viz = sol.animate(show_now=True)